<h1>Principal component analysis (PCA)</h1>

[Principal component analysis](http://en.wikipedia.org/wiki/Principal_component_analysis) (PCA) is a statistical method to find a rotation such that the first coordinate has the largest variance possible, and each succeeding coordinate in turn has the largest variance possible.

Usual initializations and the relevant imports:

In [1]:
val sparkVersion = "2.0.1"
val scalaVersion = scala.util.Properties.versionNumberString

sparkVersion: String = "2.0.1"
scalaVersion: String = "2.11.8"

In [2]:
classpath.add(
    "org.apache.spark" %% "spark-yarn" % sparkVersion,
    "org.apache.spark" %% "spark-mllib" % sparkVersion
)

146 new artifact(s)


146 new artifacts in macro
146 new artifacts in runtime
146 new artifacts in compile


In [3]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.mllib.util.MLUtils

import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.linalg.distributed.RowMatrix

import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.linalg.distributed.{CoordinateMatrix, MatrixEntry}

import org.apache.spark.sql.SparkSession
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.linalg.distributed.{CoordinateMatrix, MatrixEntry}

<tt>spark.mllib</tt> supports PCA for tall-and-skinny matrices stored in row-oriented format and any Vectors. We demonstrate how to compute principal components on a [<tt>RowMatrix</tt>](http://spark.apache.org/docs/latest/mllib-data-types.html#rowmatrix) and use them to project the vectors into a low-dimensional space in the cell below.

In [6]:
val sparkSession = SparkSession
  .builder()
  .master("local[1]")
  .appName("PCA")
  .getOrCreate()

val data = Array(
  Vectors.sparse(5, Seq((1, 1.0), (3, 7.0))),
  Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
  Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0))

val dataRDD = sparkSession.sparkContext.parallelize(data, 2)


val mat: RowMatrix = new RowMatrix(dataRDD)

// Compute the top 4 principal components.
// Principal components are stored in a local dense matrix.
val pc: Matrix = mat.computePrincipalComponents(4)

// Project the rows to the linear space spanned by the top 4 principal components.
val projected: RowMatrix = mat.multiply(pc)

(5,[1,3],[1.0,7.0])
[2.0,0.0,3.0,4.0,5.0]


sparkSession: SparkSession = org.apache.spark.sql.SparkSession@78d55545
data: Array[Vector] = Array((5,[1,3],[1.0,7.0]), [2.0,0.0,3.0,4.0,5.0], [4.0,0.0,0.0,6.0,7.0])
dataRDD: RDD[Vector] = ParallelCollectionRDD[8] at parallelize at Main.scala:50
mat: RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@36f0611a
pc: Matrix = -0.44859172075072673  -0.28423808214073987  0.08344545257592471   0.8364102009456849    
0.13301985745398526   -0.05621155904253121  0.044239792581370035  0.17224337841622106   
-0.1252315635978212   0.7636264774662965    -0.578071228563837    0.2554154886635869    
0.21650756651919933   -0.5652958773533949   -0.7955405062786798   4.858121429822393E-5  
-0.8476512931126826   -0.11560340501314653  -0.1550117891430013   -0.4533355491646027   
projected: RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@75cb21a

In HPC experiments, we will use the [NIPS 2014](https://archive.ics.uci.edu/ml/datasets/Bag+of+Words) paper along with the <b>bag of words</b> data mentioned within, but these datasets are too large for this notebook. We therefore create a small dataset from the documents:

    D1: the cat sat on the mat
    D2: the cat sat on the cat
    D3: the cat sat
    D4: the mat sat

Numbering the words

    0 the
    1 cat
    2 sat
    3 on
    4 the
    5 mat
    
the documents can be represented using the following sparse vectors

    Vectors.sparse(5, Seq((0, 2.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0))),
    Vectors.sparse(5, Seq((0, 2.0), (1, 2.0), (2, 1.0), (3, 1.0))),
    Vectors.sparse(5, Seq((0, 1.0), (1, 1.0), (2, 1.0))),
    Vectors.sparse(5, Seq((0, 1.0), (2, 1.0), (4, 1.0))))

Equally, it could be represented by triples of <tt>document_id word_id freq</tt> as follows

    1 0 2.0
    1 1 1.0
    1 2 1.0
    1 3 1.0
    1 4 1.0
    2 0 2.0
    2 1 2.0
    2 2 1.0
    2 3 1.0
    3 0 1.0
    3 1 1.0
    3 2 1.0
    4 0 1.0
    4 2 1.0
    4 4 1.0

(This conversion will come in useful for processing the bag of words data.) We now generate the principal component vectors for this dataset.

In [ ]:
val sc = sparkSession.sparkContext

// The data

val data = Array(
    Vectors.sparse(5, Seq((0, 2.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0))),
    Vectors.sparse(5, Seq((0, 2.0), (1, 2.0), (2, 1.0), (3, 1.0))),
    Vectors.sparse(5, Seq((0, 1.0), (1, 1.0), (2, 1.0))),
    Vectors.sparse(5, Seq((0, 1.0), (2, 1.0), (4, 1.0))))

val dataRDD = sc.parallelize(data)

val mat: RowMatrix = new RowMatrix(dataRDD)

// Compute the top 4 principal components.
// Principal components are stored in a local dense matrix.
val pc: Matrix = mat.computePrincipalComponents(4)

// Project the rows to the linear space spanned by the top 4 principal components.
val projected: RowMatrix = mat.multiply(pc)

<h2>Exercises</h2>

<h3>Exercise 1</h3>

Take a look at the <b>bagofwords</b> NIPS data. The format of this data is

    Number of documents
    Number of words in the vocabulary
    Total number of words in the collection
    docID wordID count
    docID wordID count
    ...
    docID wordID count
    
Initially, we need to read this data in: the steps in this would be roughly:

1) extract the number of documents, size of the vocabulary and strip off the first 3 lines
2) combine the words per document
3) create sparse vectors

Create this as a standalone program. You can use <tt>.partitions.size</tt> to check the number of partitions your data is divided into and you should keep everything as parallel as possible. You will benefit from creating a very small example to test your work, and then checking that your work scales up to the <tt>NYTIMES</tt> bagofwords data.

<h3>Exercise 2</h3>

If you try to run the PCA program from the notebook on a large dataset, it is likely to run out of memory as it attempts to construct the covariance matrix locally on the driver (and then uses [SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition) to generate the principal components). For this exercise, you are to implement an alternative method of computing principal components. For this, you need to [center your matrix](https://en.wikipedia.org/wiki/Centering_matrix) (for each word vector, this involves subtracting the mean) and then use [SVD](https://spark.apache.org/docs/2.0.2/mllib-dimensionality-reduction.html#svd-example). Implementations of this exist and you can follow this link to a Matlab implementation of [disPCA](http://www.cs.princeton.edu/~yingyul/DistributedCoresetAndPCA.zip).

You should be able to use the sparse vectors you generated in exercise 1, center these and then use SVD. A small scale example will allow you to check that your first principal components match those generated by <tt>computePrincipalComponents</tt>.

Run PCA on the full NIPS data and the NYTIMES data, varying the number of principal components generated.

<h3>Exercise 3</h3>

Run $k$-means with the generated vectors for both datasets.